# Real-Time Stop Sign Identification using Contour Detection
### Olivia Roberts, Anne Konicki, William Grieder

**Image Preprocessing**

In [229]:
#imports

import cv2
import numpy as np
from typing import List, Any

In [230]:
def display_image_and_wait(name: str, img) -> None:
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def display_images_and_wait(names: List[str], images: List[Any]) -> None:
    assert len(images) == len(names), "You must provide the same number of images and names"

    for name, img in zip(names, images):
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [231]:
# Convert image to HSV
bgr_img = cv2.imread('./assets/StopSign01.jpg')
hsv_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)

#display_images_and_wait(['Original', 'HSV'], [bgr_img, hsv_img])

In [232]:
# Binarize Image

# Best threshold appears to be 192. More testing needed

gray_img = cv2.cvtColor(hsv_img, cv2.COLOR_BGR2GRAY)
#display_image_and_wait('Black and White', gray_img)

_, bin_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY) # Store the 192 threshold version of the image for later use
#('Binarized', bin_img)

In [233]:
# Apply Gaussian Blur

### TODO: Apply a Gaussian Blur to the image after binarization.
# Analyze the images to see if the stop sign's become more visually distinct on the image.
# Use different threshold values as well. StopSign2 will be a useful image for this as part of the sign is obscured.
# See if applying the blur can potentially remove the graffiti affect. it may also be useful to apply the blur to detect ellipses rather than octagons.
# Blurring the image could make contour detection easier

blur_img = cv2.blur(bin_img, (3, 3))
#display_image_and_wait('Blurred', blur_img)

## Contour Detection

In [234]:
# Detect Octagons

# TODO: Find any octagons in the image

img_copy = bgr_img.copy()

contours, _ = cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

hexagon_contours = []
for contour in contours:
    # Approximate the contour to a polygon
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # If it doesn't have 8 sides, it isn't an octagon
    if len(approx) != 8:
        continue

    # Calculate all side lengths
    side_lengths = [
        np.linalg.norm(approx[i][0] - approx[(i + 1) % 8][0])
        for i in range(8)
    ]
    mean_length = np.mean(side_lengths)

    # Check if side lengths are approximately equal
    # Uses 0.2 as tolerance for images at odd angles
    if all(abs(length - mean_length) <= 0.2 * mean_length for length in side_lengths):
        hexagon_contours.append(approx)

# Uncomment lines to see raw contours being drawn

#cv2.drawContours(image=img_copy, contours=hexagon_contours, contourIdx=-1, color=(0, 255, 255), thickness=2)
#display_image_and_wait('Found Octagon Contour', img_copy)

In [235]:
# Bounding Box
for hexagon in hexagon_contours:
    rect = cv2.boundingRect(hexagon)
    if rect[2] < 100 or rect[3] < 100: continue
    x,y,w,h = rect
    cv2.rectangle(img_copy,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img=img_copy,text='Stop Sign',org=(x,y-20),fontFace=0,
                fontScale=0.8,color=(0,0,0), thickness=2)
display_image_and_wait('Found Stop Signs', img_copy)

cv2.imwrite("./FoundSign.png", img_copy)

True

# YOLO v5
